# Preliminaries

This notebook lets you import a gym environment and set up an agent that acts within the environment. Your tasks is to then implement some of the classical RL algorithms: Value iteration and Policy iteration. Play attention to how you are going to evaluate your agents.

First, we make sure that all dependencies are met

In [1]:
!pip install gym > /dev/null 2>&1

# Testing the Gym environments

Our next step is to import the gym package, create an environment, and make sure that we can use it.

In [2]:
%matplotlib notebook
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#create a cliff-walker
env = gym.make('CliffWalking-v0')

#set the start state
state = env.reset()
#and take some random actions
for i in range(4):
  #render the environment
  env.render()
  
  #select a random action
  action = env.action_space.sample()
  #take a step and record next state, reward and termination
  state, reward, done, _ = env.step(action)
  print("Acted: {}".format(action))
  print("State: {}".format(state))
  print("Reward: {}".format(reward))
  if done:
    #this environment only terminates once the goal is reached
    print("Done.")
    break

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

Acted: 0
State: 24
Reward: -1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Acted: 3
State: 24
Reward: -1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Acted: 1
State: 25
Reward: -1
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

Acted: 1
State: 26
Reward: -1


# Defining an agent

The next step is to define a class for our agents. We will derive from this class to later implement a Value Iteration, Policy Iteration and Monte Carlo control agent. The base class will only provide simple functionality.

In [3]:
class Agent :
  def __init__(self,env,discount_factor):
    self.env = env
    self.gamma = discount_factor
  
  def act(self, state):
    return self.env.action_space.sample() #returns a random action

  def evaluate(self):
    # now let's test our random action agent
    n_steps = 100 #number of steps per episode

    s = env.reset()
    episode_reward = 0
    
    for i in range(n_steps):
      s, r, d, _ = env.step(self.act(s))
      episode_reward += r
      if d:
        break
    return episode_reward

#test simple evaluation function
random_agent = Agent(env,0.99)
episode_reward=random_agent.evaluate()
print("Episode return {}".format(episode_reward))

Episode return -397


# Value Iteration Agent

In this section you are to implement an agent that solves the environment, using Value Iteration

In [4]:
class ValueAgent(Agent):
  def __init__(self,env,discount_factor,theta):
    super().__init__(env,discount_factor)
    #theta is an approximation error threshold
    self.theta = theta
    self.V = np.random.rand(self.env.shape[0], self.env.shape[1])
    #set terminal state to 0
    self.V[-1,-1] = 0
  
  def act(self, state): 
    #here choose action that would bring us to state with highest value
    values=[]
    for i in range(self.env.nA):
      v=0
      for j in range(len(self.env.P[state][i])):
        prob, next_state, reward, done = self.env.P[state][i][j]
        next_position = np.unravel_index(next_state, self.env.shape)
        v += prob*(reward + self.gamma*self.V[next_position])
      values.append(v)
    
    action = np.argmax(values)
    if (type(action)==np.array): print (action)
    return action

   

  def iterate(self):
    while(True):
      delta = 0.0
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        v = self.V[position]
        action = self.act(state)
        value = 0
        for j in range(len(self.env.P[state][action])):
          prob, next_state, reward, done = self.env.P[state][action][j]
          next_position = np.unravel_index(next_state, self.env.shape)
          value += prob * (reward + self.gamma*self.V[next_position])
        self.V[position] = value
        delta = max([delta, np.abs(v-self.V[position])])
      print(delta)
      if (delta < self.theta):
        print(delta)
        break

np.set_printoptions(precision=3, linewidth=200)

agent = ValueAgent(env,0.99,0.001)
print(agent.V)
#perform value iteration
agent.iterate()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))

print(agent.V)

[[0.012 0.426 0.891 0.428 0.27  0.132 0.523 0.869 0.422 0.013 0.586 0.623]
 [0.856 0.508 0.23  0.977 0.509 0.156 0.936 0.42  0.6   0.511 0.387 0.658]
 [0.515 0.822 0.62  0.746 0.189 0.141 0.102 0.682 0.759 0.351 0.834 0.27 ]
 [0.346 0.694 0.829 0.103 0.862 0.872 0.75  0.11  0.976 0.215 0.477 0.   ]]
2.762843760363978
1.5508861367848348
1.5353772754169865
1.5200235026628164
1.5048232676361883
1.489775034959826
1.474877284610228
1.4601285117641254
1.4455272266464858
1.4310719543800214
1.0691553781654815
0.9040815928148831
0.8950407768867343
0.8860903691178681
0.8483652657710703
0.0
0.0
Episode return -13
[[-13.125 -12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97 ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99 ]
 [-11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99   -1.   ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -1.      0

# Policy Iteration Agent
Follow the same procedure for implementing a policy iteration agent

In [5]:
class PolicyAgent(Agent):
  def __init__(self,env,discount_factor,theta):
    super().__init__(env,discount_factor)
    #theta is an approximation error threshold
    self.theta = theta
    self.V = np.random.rand(self.env.shape[0], self.env.shape[1])
    #set terminal state to 0
    self.V[-1,-1] = 0
    self.policy = np.random.randint(4, size=self.env.shape)
  
  def act(self, state): 
    #here choose action that would bring us to state with highest value
    values=[]
    for i in range(self.env.nA):
      v=0
      for j in range(len(self.env.P[state][i])):
        prob, next_state, reward, done = self.env.P[state][i][j]
        next_position = np.unravel_index(next_state, self.env.shape)
        v += prob * (reward + self.gamma*self.V[next_position])
      values.append(v)
    
    action = np.argmax(values)
    if (type(action)==np.array): print (action)
    return action

   

  def evaluate_policy(self):
    while(True):
      delta = 0.0
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        v = self.V[position]
        action = self.policy[position]
        value = 0
        for j in range(len(self.env.P[state][action])):
          prob, next_state, reward, done = self.env.P[state][action][j]
          next_position = np.unravel_index(next_state, self.env.shape)
          value += prob * (reward + self.gamma*self.V[next_position])
        self.V[position] = value
        delta = max([delta, np.abs(v-self.V[position])])
      #print(delta)
      if (delta < self.theta):
        #print(delta)
        break

  def improve(self):
    policy_stable = True
    #for i in range(200):
    while(True):
      policy_stable = True
      for state in range(self.env.nS-1):
        position = np.unravel_index(state, self.env.shape)
        old_action = self.policy[position]
        self.policy[position] = self.act(state)
        if not (self.policy[position] == old_action):
          policy_stable = False
      if policy_stable:
        break
      else:
        self.evaluate_policy()





agent = PolicyAgent(env,0.99,0.001)
print(agent.V)

print(agent.policy)
#perform value iteration
agent.improve()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))
np.set_printoptions(precision=3, linewidth=200)
print(agent.V)

print(agent.policy)

[[0.641 0.342 0.921 0.84  0.38  0.566 0.278 0.218 0.428 0.755 0.357 0.673]
 [0.378 0.439 0.53  0.379 0.904 0.24  0.987 0.418 0.026 0.232 0.879 0.325]
 [0.093 0.607 0.09  0.883 0.95  0.532 0.496 0.729 0.315 0.114 0.593 0.194]
 [0.689 0.934 0.675 0.865 0.717 0.578 0.886 0.177 0.922 0.817 0.068 0.   ]]
[[0 1 1 1 1 0 1 1 0 1 3 0]
 [2 3 3 0 0 2 0 1 1 2 0 3]
 [0 3 2 1 1 0 2 1 1 3 2 2]
 [1 2 1 1 3 1 2 3 3 3 1 1]]
Episode return -13
[[-13.125 -12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97 ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99 ]
 [-11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -2.97   -1.99   -1.   ]
 [-12.248 -11.362 -10.466  -9.562  -8.648  -7.726  -6.793  -5.852  -4.901  -3.94   -1.      0.   ]]
[[1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 1]]


In [6]:
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
import copy

class NonDCliffWalkingEnv(CliffWalkingEnv):
  def __init__(self):
    super(NonDCliffWalkingEnv, self).__init__()
  
  def _calculate_transition_prob(self, current, delta):
    """
    Determine the outcome for an action. Transition Prob is always 1.0.
    :param current: Current position on the grid as (row, col)
    :param delta: Change in position for transition
    :return: (1.0, new_state, reward, done)
    """
    transitions = []
    terminal_state = (self.shape[0] - 1, self.shape[1] - 1)
    deltas = [delta, [2*delta[0], 2*delta[1]]]
    zero_index = delta.index(0)
    delta_off = copy.copy(delta)
    delta_off[zero_index] = 1
    deltas.append(copy.copy(delta_off))
    delta_off[zero_index] = -1
    deltas.append(delta_off)
    probabilities = [0.85,0.05,0.05,0.05]

    
    for d,p in zip(deltas, probabilities):
      new_position = np.array(current) + np.array(d)
      new_position = self._limit_coordinates(new_position).astype(int)
      new_state = self.start_state_index if self._cliff[tuple(new_position)] else np.ravel_multi_index(tuple(new_position), self.shape)
      reward = -100 if self._cliff[tuple(new_position)] else -1
      is_done = tuple(new_position) == terminal_state
      #print(tuple(new_position), self._cliff[tuple(new_position)])
      transitions.append((p, new_state, reward, is_done))
    
    return transitions
  

In [7]:
penv = NonDCliffWalkingEnv()

In [8]:
agent = ValueAgent(penv,0.99,0.001)
print(agent.V)
#perform value iteration
agent.iterate()
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))

print(agent.V)

[[0.365 0.873 0.254 0.386 0.905 0.495 0.214 0.012 0.917 0.313 0.624 0.962]
 [0.64  0.148 0.632 0.914 0.171 0.167 0.356 0.038 0.844 0.703 0.076 0.105]
 [0.963 0.891 0.544 0.507 0.809 0.875 0.84  0.532 0.345 0.011 0.23  0.797]
 [0.427 0.947 0.297 0.3   0.083 0.327 0.706 0.53  0.624 0.683 0.327 0.   ]]
3.4466326055733845
1.6689345116641454
1.5848090408697173
1.3895449618879923
1.3238920575528157
1.2652613471889147
1.2127562406416956
1.1668226757445392
1.1231294286688218
0.9673417294492275
0.955859307854741
0.9446285087963702
0.9336304871008121
0.9228475972050365
0.9122625171248924
0.9018570717321328
0.8861190025830403
0.8760510896740605
0.8584305423870155
0.7222133959945438
0.22571812050809115
0.05591775103848562
0.04002370896781571
0.03922236717693828
0.03882195580299985
0.03842435816424761
0.03802960637174202
0.03763823595917515
0.037248527336988246
0.03681507782210147
0.036032019780645186
0.033349938634394505
0.025232703464631356
0.011714489190701727
0.002874895295381208
0.001686001751

In [9]:
pagent = PolicyAgent(penv,0.99,0.001)
print(agent.V)

print(pagent.policy)
#perform value iteration
pagent.improve()
#evaluate agent and plot relevant qualities
episode_reward=pagent.evaluate()
print("Episode return {}".format(episode_reward))

print(pagent.V)

print(pagent.policy)

[[-17.382 -16.593 -15.798 -14.999 -14.196 -13.389 -12.578 -11.765 -10.949 -10.132  -9.31   -8.445]
 [-17.326 -16.486 -15.635 -14.773 -13.897 -13.009 -12.108 -11.192 -10.262  -9.31   -8.445  -7.504]
 [-18.114 -17.326 -16.486 -15.635 -14.773 -13.897 -13.01  -12.108 -11.192 -10.262  -7.504  -6.883]
 [-18.855 -18.109 -17.279 -16.436 -15.581 -14.714 -13.834 -12.941 -12.034 -11.022  -1.341   0.   ]]
[[2 1 1 0 0 2 3 3 2 2 2 3]
 [2 2 2 2 3 0 2 1 1 3 0 3]
 [2 0 1 2 2 0 3 2 1 3 0 2]
 [2 1 1 1 0 0 3 2 1 2 1 2]]
Episode return -15
[[-17.384 -16.594 -15.8   -15.001 -14.198 -13.391 -12.58  -11.766 -10.95  -10.133  -9.311  -8.446]
 [-17.328 -16.488 -15.637 -14.774 -13.899 -13.011 -12.109 -11.194 -10.263  -9.311  -8.446  -7.505]
 [-18.116 -17.328 -16.488 -15.637 -14.774 -13.899 -13.011 -12.109 -11.194 -10.263  -7.505  -6.883]
 [-18.857 -18.111 -17.281 -16.437 -15.582 -14.715 -13.836 -12.942 -12.035 -11.023  -1.341   0.   ]]
[[1 1 1 1 1 1 1 1 1 1 2 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 

#Monte Carlo control agent
Follow the same procedure for implementing a Monte Carlo control agent

In [62]:
class MonteCarloAgent(Agent):
  def __init__(self,env,discount_factor,epsilon):
    super().__init__(env,discount_factor)
    #theta is an approximation error threshold
    self.epsilon = epsilon
    self.Q = np.random.rand(self.env.shape[0], self.env.shape[1], self.env.nA)
    #set terminal state to 0
    self.Q[-1,-1,:] = 0
    self.policy = np.random.rand(self.env.shape[0], self.env.shape[1], self.env.nA)
    # Normalize to 1.0
    for i in range(self.env.shape[0]):
      for j in range(self.env.shape[1]):
        self.policy[i,j,:] /= np.sum(self.policy[i,j,:].flatten())
    self.returns = np.empty((self.env.shape[0], self.env.shape[1], self.env.nA), dtype=list)
    for i in range(self.env.shape[0]):
      for j in range(self.env.shape[1]):
        for k in range(self.env.nA):
          self.returns[i,j,k] = []
    self.first_visits = np.full((self.env.shape[0], self.env.shape[1], self.env.nA), -1, dtype=np.int)
  
  def explore(self, state): 
    #here choose action according to the epsilon-soft policy
    position = np.unravel_index(state, self.env.shape)
    sample = np.random.rand()
    sum = 0.
    for action,p in enumerate(self.policy[position[0], position[1],:].flatten()):
      sum += p
      if sample <= sum:
        break
    return action

  def act(self, state): 
    #here choose action according to the epsilon-soft policy
    position = np.unravel_index(state, self.env.shape)
    action = np.argmax(self.policy[position[0], position[1],:].flatten())
    return action

  def run_one_episode(self, exploring_starts):
    icliff = 0 
    episode = []
    done = False
    reward = 0
    self.env.reset()
    if exploring_starts:
      state = np.random.randint(0,37)
      action = np.random.randint(0,self.env.nA)
      self.env.s = state
    else:
      state = self.env.s
      action = self.explore(state)
      
    istep = 0
    while (not done):
      new_state, reward, done, _ = self.env.step(action)      
      episode.append((state, action, reward))
      position = np.unravel_index(state, self.env.shape)
      if self.first_visits[position[0], position[1],action] == -1: 
        self.first_visits[position[0], position[1],action] = istep
      new_position = np.unravel_index(new_state, self.env.shape)
      if exploring_starts:
        done = done or reward == -100 #self.env._cliff[tuple(new_position)]
      istep += 1
      if reward == -100: #(self.env._cliff[tuple(new_position)]):
        icliff +=1 #print('Fell off the cliff')
      state = self.env.s
      action = self.explore(state)
    print('Fell off the cliff ' + str(icliff) + ' times')
    return episode

  def iterate_policy(self, n, exploring_starts = False, epsilon=None):
    if epsilon is not None:
      self.epsilon = epsilon

    for i in range(n):
      self.first_visits = np.full((self.env.shape[0], self.env.shape[1], self.env.nA), -1, dtype=np.int)
      episode = self.run_one_episode(exploring_starts)
      G = 0
      T = len(episode)
      print('Episode length: ' + str(T))
      for t in reversed(range(T)):
        (state, action,  reward) = episode[t]
        G = self.gamma * G + reward
        position = np.unravel_index(state, self.env.shape)
        if (self.first_visits[position[0], position[1], action] == t):
          self.returns[position[0], position[1], action].append(G)
          self.Q[position[0], position[1], action] = np.mean(self.returns[position[0], position[1], action])
          A_star = np.argmax(self.Q[position[0], position[1], :])
          for ip in range(self.env.nA): 
            self.policy[position[0], position[1], ip] = 1 - self.epsilon + self.epsilon / self.env.nA if ip == A_star else self.epsilon / self.env.nA
      print('Iteration ' + str(i) + ' done.')

In [ ]:
agent = MonteCarloAgent(env,0.99,0.2)
#perform value iteration
agent.iterate_policy(100, True)

In [ ]:
agent.iterate_policy(1000,False, 0.05)

In [69]:
#evaluate agent and plot relevant qualities
episode_reward=agent.evaluate()
print("Episode return {}".format(episode_reward))

print(agent.Q[3,0])
print(np.argmax(agent.policy, axis=2))

Episode return -17
[ -17.194 -103.901  -20.081  -22.923]
[[1 1 1 1 1 1 1 1 1 1 2 2]
 [1 0 3 0 0 3 0 1 1 1 1 2]
 [0 3 0 0 0 1 0 0 0 0 1 2]
 [0 0 1 3 3 3 1 3 1 0 2 3]]


In [ ]:

pagent = MonteCarloAgent(penv,0.99,0.2)
#perform value iteration
pagent.iterate_policy(100)

In [ ]:
pagent.iterate_policy(1000, 0.1)

In [73]:
#evaluate agent and plot relevant qualities
episode_reward=pagent.evaluate()
print("Episode return {}".format(episode_reward))

print(pagent.Q[1,0])
print(np.argmax(pagent.policy, axis=2))

Episode return -100
[-126.333 -126.803 -153.514 -110.973]
[[1 0 1 2 2 3 1 0 2 1 2 3]
 [3 0 0 3 1 3 1 1 1 0 2 0]
 [0 3 1 1 0 1 0 0 3 0 1 2]
 [0 2 2 1 2 3 3 3 3 1 2 2]]
